In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv


load_dotenv()

loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000416134",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body","media_end_head go_trans"]}
        )
    )
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

splits = text_splitter.split_documents(docs)

vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriver = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
"""
당신은 질문-답변을 수행하는 AI 어시스턴트이다.
주어진 문맥에 검색된 다음문맥(context)를 사용해 질문에 답하세요
만약 주어진 문멕에서 답을 찾을 수 없을경우, 모른다고 이야기하세요.
한글로 답변해주세요.

#Question
{question}

#Context:
{context}

#Answer:"""
)

llm = ChatOpenAI(model="gpt-4", temperature=0)

news_chain = (
    {"context":retriver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = news_chain.invoke("블랙핑크는 몇위?")

print(answer)

블랙핑크는 '아이스크림(Ice Cream)'이라는 곡으로 빌보드 '핫100'에서 13위를 기록했습니다.


In [7]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/snow-white.pdf")

docs = loader.load()

print(f"문서의 페이지수 :{len(docs)}")

문서의 페이지수 :6


In [11]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/snow-white.pdf")

docs = loader.load()

print(docs[5].page_content)

왕자는깨어난백설공주를보고기뻐했어요.
“공주님, 나는이웃나라왕자입니다.”
“왕자님이나를다시살려주셨군요.”
“나와결혼해주시겠어요?”
“네, 좋아요!”
두사람은일곱난쟁이와함께오래오래행복하게살
았답니다.



In [12]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/snow-white.pdf")

docs = loader.load()

# 메타데이터
print(docs[0].__dict__)

{'id': None, 'metadata': {'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, 'page_content': '백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.\n하지만기쁨도잠시, 왕비는곧세상을떠나고말았어\n요.\n', 'type': 'Document'}


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의수 : {len(split_documents)}")

분할된 청크의수 : 21


In [ ]:
# 임베딩생성
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [19]:
#db생성 및 저장
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

for doc in vectorstore.similarity_search("왕자"):
    print(doc.page_content)
    
# 벡터스토어에 있는 정보를 검색하고 생성
retriver = vectorstore.as_retriever()

retriver.invoke("백설공주")

왕자는깨어난백설공주를보고기뻐했어요.
“공주님, 나는이웃나라왕자입니다.”
“왕자님이나를다시살려주셨군요.”
“나와결혼해주시겠어요?”
“네, 좋아요!”
왕자는백설공주에게반해유리관을달라고부탁했어요.
일곱난쟁이는백설공주를잘지킨다는약속을받고유리관을
내주었지요.
그런데신하들이유리관을옮기다돌부리에툭! 백설공주목
에서사과조각이툭!
하지만기쁨도잠시, 왕비는곧세상을떠나고말았어
요.
북적이는소리에잠이깬백설공주는왕비를피해도망쳤다고
이야기했어요.
“불쌍한공주님, 우리와함께살아요. 조심조심또조심. 낯선
사람에게는문을열어주지마세요.”


[Document(metadata={'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, page_content='백설공주\n옛날어느왕국에공주님이태어났어요.\n“어쩜이렇게어여쁠까? 살결이눈처럼하얗구나. 백\n설공주라고불러야겠다.”\n왕과왕비는갓태어난딸을보며기뻐했어요.'),
 Document(metadata={'source': 'data/snow-white.pdf', 'file_path': 'data/snow-white.pdf', 'page': 3, 'total_pages': 6, 'format': 'PDF 1.5', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'producer': 'Microsoft® PowerPoint® 2013', 'creationDate': "D:20230912112024+09'00'", 'modDate': "D:20230912112024+09'00'", 'trapped': ''}, page_content='간사과랍니다. 잠깐문을열어보세요.”\n백설공주는고개를저었어요.\n“난쟁이들이문을열어주지말라고했어요.”\n백설공주가거절하자, 왕비는창문틈새로사과를쑥내밀었어\n

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    """
당신은 질의 응답 작업의 보조자입니다. 
다음 검색된 컨텍스트 조각을 사용하여 질문에 답하세요.
마치 유치원 선생님이 아이에게 말을 거는 것처럼 매우 친절하고 온화한 어조를 사용하십시오.
따뜻하고 친근하게 말하세요.
답을 모르면 모른다고 하면 됩니다.
한국어로 대답하세요

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

llm = ChatOpenAI(model="gpt-4o", temperature=0)

chain = (
    {"context": retriver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "신데렐라는 유리구두를 언제잃어버렸어?"
response = chain.invoke(question)

print(response)

미안하지만, 신데렐라가 유리구두를 잃어버린 이야기는 여기에는 없어요. 다른 이야기를 찾아봐야 할 것 같아요.
